Contract details
-------------------

In [7]:
from ib_insync import *
util.startLoop()

import logging
# util.logToConsole(logging.DEBUG)

ib = IB()
ib.connect('127.0.0.1', 4002, clientId=11)

<IB connected to 127.0.0.1:4002 clientId=11>

In [24]:
intc = Stock('AAPL', 'SMART', 'USD')
ib.qualifyContracts(intc)

[Stock(conId=265598, symbol='AAPL', exchange='SMART', primaryExchange='NASDAQ', currency='USD', localSymbol='AAPL', tradingClass='NMS')]

In [16]:
contract = Index('NDX', 'NASDAQ', 'USD')
ib.qualifyContracts(contract)

[Index(conId=416843, symbol='NDX', exchange='NASDAQ', currency='USD', localSymbol='NDX')]

Suppose we want to find the contract details for AMD stock.
Let's create a stock object and request the details for it:

In [17]:
amd = Index('NDX', 'NASDAQ', 'USD')

cds = ib.reqContractDetails(amd)

len(cds)

1

We get a long list of contract details. Lets print the first one:

In [18]:
cds[0]

ContractDetails(contract=Contract(secType='IND', conId=416843, symbol='NDX', exchange='NASDAQ', currency='USD', localSymbol='NDX'), marketName='', minTick=0.01, orderTypes='ACTIVETIM,AD,ADJUST,ALERT,ALLOC,BASKET,BENCHPX,COND,CONDORDER,DAY,DEACT,DEACTDIS,DEACTEOD,GAT,GTC,GTD,GTT,HID,LMT,NONALGO,OCA,SCALE,SCALERST,WHATIF', validExchanges='NASDAQ', priceMagnifier=1, underConId=0, longName='NASDAQ 100 Stock Index', contractMonth='', industry='Indices', category='Broad Range Equity Index', subcategory='*', timeZoneId='US/Eastern', tradingHours='20240729:0930-20240729:1600;20240730:0930-20240730:1600;20240731:0930-20240731:1600;20240801:0930-20240801:1600;20240802:0930-20240802:1600', liquidHours='20240729:0930-20240729:1600;20240730:0930-20240730:1600;20240731:0930-20240731:1600;20240801:0930-20240801:1600;20240802:0930-20240802:1600', evRule='', evMultiplier=0, mdSizeMultiplier=1, aggGroup=2147483647, underSymbol='', underSecType='', marketRuleIds='25', secIdList=[], realExpirationDate='',

The contract itself is in the 'contract' property of the contract details. Lets make a list of contracts and look at the first:

In [19]:
contracts = [cd.contract for cd in cds]

contracts[0]

Contract(secType='IND', conId=416843, symbol='NDX', exchange='NASDAQ', currency='USD', localSymbol='NDX')

To better spot the difference between all the contracts it's handy to convert to a DataFrame. There is a utility function to do that:

In [20]:
util.df(contracts)

,secType,conId,symbol,lastTradeDateOrContractMonth,strike,right,multiplier,exchange,primaryExchange,currency,localSymbol,tradingClass,includeExpired,secIdType,secId,description,issuerId,comboLegsDescrip,comboLegs,deltaNeutralContract
0,IND,416843,NDX,,0.0,,,NASDAQ,,USD,NDX,,False,,,,,,[],None


We can see from this that AMD trades in different currencies on different exchanges.
Suppose we want the one in USD on the SMART exchange. The AMD contract is adjusted to
reflect that and becomes unique:

In [6]:
amd = Stock('AMD', 'SMART', 'USD')

assert len(ib.reqContractDetails(amd)) == 1

Lets try the same for Intel:

In [7]:
intc = Stock('INTC', 'SMART', 'USD')

assert len(ib.reqContractDetails(intc)) == 1

Let's try a non-existing contract:

In [8]:
xxx = Stock('XXX', 'SMART', 'USD')

assert len(ib.reqContractDetails(xxx)) == 0

Error 200, reqId 6: No security definition has been found for the request, contract: Stock(symbol='XXX', exchange='SMART', currency='USD', comboLegs=[])


or a Forex contract

In [9]:
eurusd = Forex('EURUSD')

assert len(ib.reqContractDetails(eurusd)) == 1

With the ``qualifyContracts`` method the extra information that is send back
from the contract details request is used to fill in the original contracts.

Lets do that with ``amd`` and compare before and aftwards:

In [10]:
amd

Stock(symbol='AMD', exchange='SMART', currency='USD', comboLegs=[])

In [21]:
ib.qualifyContracts(amd)
amd

Index(conId=416843, symbol='NDX', exchange='NASDAQ', currency='USD', localSymbol='NDX')

**TIP:** When printing a contract, the output can be copy-pasted and it will be valid Python code.

The ``conId`` that is returned can by itself be used to uniquely specify a contract:

In [22]:
contract_4391 = Contract(conId=416843)

ib.qualifyContracts(contract_4391)

assert contract_4391 == amd

A whole bunch of contracts can be qualified at the same time. A list of all the successfull ones is returned:

In [23]:
qualContracts = ib.qualifyContracts(amd, intc, xxx, eurusd)

assert intc in qualContracts
assert xxx not in qualContracts

NameError: name 'intc' is not defined

There is also an API function to request stocks (only stocks) that match a pattern:

In [14]:
matches = ib.reqMatchingSymbols('intc')
matchContracts = [m.contract for m in matches]

matches

[ContractDescription(contract=Contract(secType='STK', conId=38709539, symbol='INTC', primaryExchange='MEXI', currency='MXN', comboLegs=[]), derivativeSecTypes=[]),
 ContractDescription(contract=Contract(secType='STK', conId=270639, symbol='INTC', primaryExchange='NASDAQ.NMS', currency='USD', comboLegs=[]), derivativeSecTypes=['CFD', 'OPT', 'IOPT', 'WAR', 'FUT']),
 ContractDescription(contract=Contract(secType='STK', conId=12178234, symbol='INTC', primaryExchange='EBS', currency='CHF', comboLegs=[]), derivativeSecTypes=[]),
 ContractDescription(contract=Contract(secType='STK', conId=11463493, symbol='INTCUSD', primaryExchange='EBS', currency='USD', comboLegs=[]), derivativeSecTypes=[]),
 ContractDescription(contract=Contract(secType='STK', conId=130861844, symbol='S', primaryExchange='MEXI', currency='MXN', comboLegs=[]), derivativeSecTypes=[]),
 ContractDescription(contract=Contract(secType='STK', conId=130861851, symbol='2S7', primaryExchange='FWB', currency='EUR', comboLegs=[]), deri

In [15]:
assert intc in matchContracts

In [6]:
ib.disconnect()